In [37]:
from tensor import Tensor
import numpy as np

In [31]:
x = Tensor([[1], [2], [3], [4], [5]])

kernel = Tensor([[1], [2], [3]])

y = x.conv1d(kernel)

y

Tensor:(data = [14. 20. 26.])

In [10]:
# suppose a 1d signal S and kernal k
S = np.array([2, 3, 1, 2, 3])
k = np.array([2, 1, 1])

# perform 1d conv to get result R
# R should be same as k if no padding
# i = 0
# j = len(k)
# R = []
# while j <= len(S):
#     prod = np.dot(S[i:j], k)
#     R.append(prod)
#     i += 1
#     j += 1
# R = np.array(R)
# R

# "In general, if we have an N×N filter, a W×H image, and stride S there are (W-N) //S + 1 places to apply the filter ..."

S = Tensor([[1], [2], [3], [4], [5]]) # (5, 1)
S2 = Tensor([1, 2, 3, 4, 5]) 
k = Tensor([[1], [2], [3]]) # (3, 1)
k2 = Tensor([1, 2, 3]) 
s = 1 # stride
R_shape = (S.shape[0] - k.shape[0]) // s + 1
R = Tensor(np.zeros(R_shape))
R2 = Tensor(np.zeros(R_shape))
# for the conv operation, slide k over S
for i in range(0, S.shape[0] - k.shape[0] + 1, s):
    R.data[i] = (S.data[i: i+k.shape[0]] * k.data).sum()

for i in range(0, S2.shape[0] - k2.shape[0] + 1, s):
    R2.data[i] = (S2.data[i: i+k2.shape[0]] * k2.data).sum()
    
R2

Tensor:(data = [14. 20. 26.])

In [66]:
# Implementing a 2d convolution for conv2d layer

S3 = Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
k3 = Tensor([[1, 2], [3, 4]])
fm_shape = ((S3.shape[0] - k3.shape[0]) // s + 1, (S3.shape[1] - k3.shape[1]) // s + 1)
fm = Tensor(np.zeros(fm_shape))
# fm.data[0][0] = (S3.data[0:2, 0:2] * k3.data).sum()
# fm.data[0][1] = (S3.data[0:2, 1:3] * k3.data).sum()
# fm.data[1][0] = (S3.data[1:3, 0:2] * k3.data).sum()
# fm.data[1][1] = (S3.data[1:3, 1:3]  * k3.data).sum()

# fm.data[i][j] = (S3.data[i:i+k3.shape[0], j:j+k3.shape[0]] * k3.data).sum()

# the indexing seems to be working
for i in range(0, fm_shape[1], s):
    for j in range(0, fm_shape[0], s):
        fm.data[i][j] = (S3.data[i:i+k3.shape[0], j:j+k3.shape[0]] * k3.data).sum()
        
# S3.data[0:2, 1:3], S3.data[1:3, 0:2], S3.data[1:3, 1:3]
fm

Tensor:(data = [[37. 47.]
 [67. 77.]])

In [72]:
def conv2dd(signal, kernel, stride=1):
    out_shape = ((signal.shape[0] - kernel.shape[0]) // stride + 1, (signal.shape[1] - kernel.shape[1]) // stride + 1)
    output = Tensor(np.zeros(out_shape))
    # fm.data[0][0] = (S3.data[0][0:2] * k3.data[0]).sum() + (S3.data[1][0:2] * k3.data[1]).sum()
    # fm.data[0][1] = (S3.data[0][1:3] * k3.data[0]).sum() + (S3.data[1][1:3] * k3.data[1]).sum()
    # fm.data[1][0] = (S3.data[1][0:2] * k3.data[0]).sum() + (S3.data[2][0:2] * k3.data[1]).sum()
    # fm.data[1][1] = (S3.data[1][1:3] * k3.data[0]).sum() + (S3.data[2][1:3] * k3.data[1]).sum()

    #fm.data[i][j] = (S3.data[i][j:j+k3.shape[0]] * k3.data[0]).sum() + (S3.data[i+1][j:j+k3.shape[0]] * k3.data[1]).sum()

    # the indexing seems to be working, hardcoded for a 2x2 filter/kernel
    for i in range(0, out_shape[1], stride):
        for j in range(0, out_shape[0], stride):
            output.data[i][j] = (signal.data[i:i+kernel.shape[0], j:j+kernel.shape[0]] * kernel.data).sum()
    return output
signal = Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
kernel = Tensor([[1, 2], [3, 4]])

feature_map = conv2dd(signal, kernel)
fm2 = conv2dd(signal, kernel)
feature_map

Tensor:(data = [[37. 47.]
 [67. 77.]])

In [71]:
signal = Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
kernel = Tensor([[1, 2], [3, 4]])

feature_map = conv2dd(signal, kernel)
fm2 = conv2dd(signal, kernel)
feature_map

Tensor:(data = [[0. 0.]
 [0. 0.]])

In [39]:
a = np.array([[1], [2], [3]])
b = np.array([[1], [2], [3], [4], [5]])
a.shape, b.shape, b[0]
shp = (b.shape[0] - a.shape[0]) + 1
c = np.zeros(shp)
c += (b[0:3] * a).sum()
c

array([14., 14., 14.])

In [ ]:
def conv1d(self, kernel, stride=1):
    # Initialize the output tensor with the appropriate shape
    output_shape = (self.shape[0] - kernel.shape[0]) // stride + 1
    output = Tensor(np.zeros(output_shape), [self, kernel])

    # Slide the kernel over the input tensor and perform the convolution operation
    for i in range(0, self.shape[0] - kernel.shape[0] + 1, stride):
        output.data[i] = (self.data[i:i+kernel.shape[0]] * kernel.data).sum()

    # Define the backward pass for the convolution operation
    def backward():
        # Compute the gradients of the input and kernel tensors
        self.grad += kernel.data * output.grad
        kernel.grad += self.data[:output.shape[0]] * output.grad

        # Zero out the gradients of the padded elements
        self.grad[kernel.shape[0]:] = 0
    output.backward = backward

    return output


# Create an input tensor with shape (5, 1)
x = Tensor([[1], [2], [3], [4], [5]])

# Create a kernel tensor with shape (3, 1)
kernel = Tensor([[1], [2], [3]])

# Perform the convolution with stride size 1
y = x.conv1d(kernel)

# Print the output tensor
print(y)
